# YOU NEED TO UPLOAD THESE FILES FOR THINGS TO WORK:
# - TRAIN SET: 'training_converted.csv'
# - TEST SET: 'test_set.pkl'

note 1: these are pre-reformatted data files. Here, sentences are formatted as:
- sentence - explanation A - label
- sentence - explanation B - label
- sentence - explanation C - label


as opposed to the original format of:
- sentence - explanation A - explanation B - explanation C - label

note 2: this notebook is an adaption of the following notebook on single-text sentiment analysis of movie reviews, see: https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb

We adapted the notebook such that the model takes as an input not one, but two texts, and generates predictions accordingly. Our notebook also includes some auxiliary functions to label sentences with the most likely explanation.

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

import numpy as np
import re
import random
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('seaborn')
pd.set_option('display.max_colwidth', 100)

     |████████████████████████████████| 71kB 3.6MB/s 



In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'OUTPUT_DIR_NAME'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: OUTPUT_DIR_NAME *****


In [0]:
#THIS IS JUST A FUNCTION TO MAKE SURE EACH SENTENCE ONLY HAS ONE EXPLANATION RATHER THAN 0, 2 OR 3!
def final_solution(label_list, true_list):
  
  #this function goes over all the lists and returns a list which can be subtracted from the original list
  final_list = []

  #iterate over length of dataframe in steps of 3
  for i in range(0, len(label_list), 3):

    #take the sum of the current, second and third label
    label_sum = sum(label_list[i:i+3])

    #lets go over the cases 1 by 1

    #nothing wrong, so we just add a bunch of 0 (3 to be exact). if we subtract these, nothing happens with the original list!
    if label_sum == 1:
      final_list.append(0)
      final_list.append(0)
      final_list.append(0)

    #if this is true, apparently all predictions suck
    elif label_sum == 0:

      #lets add the three explanations to a list:
      exp_list = []
      exp_list.append(true_list[i])
      exp_list.append(true_list[i+1])
      exp_list.append(true_list[i+2])

      #we find the most true value and its index in the list
      most_prob = max(exp_list)
      most_prob_ind = exp_list.index(most_prob)
      
      #create an empty list which we'll append to our final_list
      to_be_appended = [0,0,0]

      #edit the value at the index of the most true value. we're subtracting the final_list from the main list, which is why we use -1
      to_be_appended[most_prob_ind] = -1

      final_list = final_list + to_be_appended
    

    #if this is true, then two explanations make sense, but one needs to go -> the second most true one, specifically.
    elif label_sum == 2:

      #lets add the three explanations to a list:
      exp_list = []
      exp_list.append(true_list[i])
      exp_list.append(true_list[i+1])
      exp_list.append(true_list[i+2])

      #sort list by descending order, such that we know that the second most true value is at index 1
      dum_list = sorted(exp_list, reverse=True)
      second_most_prob = dum_list[1]

      #find index of that value in our original list
      second_most_prob_ind = exp_list.index(second_most_prob)
      
      #create an empty list which we'll append to our final_list
      to_be_appended = [0,0,0]

      #add a -1 at the index of our second most prob. again, we are going to subtract this list, so we need to have a 1 there if we want to get rid of something
      to_be_appended[second_most_prob_ind] = 1

      final_list = final_list + to_be_appended


    #if this is true, then three explanations make sense
    elif label_sum == 3:

      #lets add the three explanations to a list:
      exp_list = []
      exp_list.append(true_list[i])
      exp_list.append(true_list[i+1])
      exp_list.append(true_list[i+2])

      #sort list by descending order, such that we know that the second most true value is at index 1, and the 3rd most prob at index 2
      dum_list = sorted(exp_list, reverse=True)
      second_most_prob = dum_list[1]
      third_most_prob = dum_list[2]

      second_most_prob_ind = exp_list.index(second_most_prob)
      third_most_prob_ind = exp_list.index(third_most_prob)

      #create an empty list which we'll append to our final_list
      to_be_appended = [0,0,0]

      #add a -1 at the index of our second most prob
      to_be_appended[second_most_prob_ind] = 1
      to_be_appended[third_most_prob_ind] = 1

      final_list = final_list + to_be_appended
  
  return final_list

#Data

Here we load the training data for the sentiment classification. We use the first 16.000 sentences for training, and the remaining 8000 sentences for testing.

In [0]:
train = pd.read_csv('training_converted.csv')
train = train[['FalseSent', 'Option', 'label']]
test = train.iloc[16001:]
train = train.iloc[:16001]


For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [0]:
DATA_COLUMN = 'FalseSent'
OPTION_COLUMN = 'Option'
LABEL_COLUMN = 'label'
# label_list is the list of labels, i.e. True, False or 0, 1
label_list = [0, 1]

#Data Preprocessing
here, we convert our training data into Python objects that BERT understands
- `text_a` is the false sentence.
- `text_b` is the explanation
- `label` is the label for our example, i.e. True, False

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[OPTION_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[OPTION_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Tokenizer function to lowercase and tokenize sentences.


In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


InputExamples are fed into `run_classifier.convert_examples_to_features`, converting them into features BERT understands.

In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 16001


INFO:tensorflow:Writing example 0 of 16001


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


#Creating a model



In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Wrapping our model function in a `model_fn_builder` function.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 2.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f236333a898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'OUTPUT_DIR_NAME', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f236333a898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:loss = 0.4305341, step = 1100 (86.826 sec)


INFO:tensorflow:global_step/sec: 1.15142


INFO:tensorflow:global_step/sec: 1.15142


INFO:tensorflow:loss = 0.17559166, step = 1200 (86.852 sec)


INFO:tensorflow:loss = 0.17559166, step = 1200 (86.852 sec)


Now, let's run the model on the unseen test set, but this time, only extract the predictions, and do not compute scores.

The cell below should take about a minute to run.


In [0]:
test_data = pd.read_pickle('test_set.pkl')
test_data['Label'] = 0

DATA_COLUMN = 'FalseSent'
OPTION_COLUMN = 'Option'
LABEL_COLUMN = 'Label'

# test_data = test_data.head(100)

# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]


#convert test_data to input_examples
test_data_InputExamples = test_data.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[OPTION_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

#convert input_examples to features
test_data_features = bert.run_classifier.convert_examples_to_features(test_data_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

#build the input function
test_data_input_fn = run_classifier.input_fn_builder(
    features=test_data_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

#predict and store resulting generator variable in predictions
predictions = estimator.predict(test_data_input_fn)

#store predictions in list
prediction_list = list(predictions)

#for loop to convert list of dictionary with labels to just a list of labels and a list of true/false probabilities
pred_list = []
prob_true_list = []
prob_false_list = []
for prediction in prediction_list:
  pred_list.append(prediction['labels'])
  prob_true_list.append(prediction['probabilities'][0])
  prob_false_list.append(prediction['probabilities'][1])

#for some reason the probabilies are zero, idk either why this is the case. function to overcome that:
def negativ(mylist):
  myneglist = [ -x for x in mylist]
  return myneglist

#now we convert the probabilities into nice, positive values as they should be
prob_true_list = negativ(prob_true_list)
prob_false_list = negativ(prob_false_list)

#overwrite Label in test_data, add column Probabilities
test_data['Label'] = pd.Series(pred_list)
test_data['Prob True'] = pd.Series(prob_true_list)
test_data['Prob False'] = pd.Series(prob_false_list)

test_data





Now we run a function that determines which option is the most true one, such that each sentence has one and only one explanation assigned to it - rather than 0, or 2, or 3 explanations.

In [0]:
test_data.tail(50)

# test_data = pd.read_pickle('test_set.pkl')
# test_data.head(50)

In [0]:
final_list = final_solution(pred_list, prob_true_list)

# final_list

test_data['correction'] = pd.Series(final_list)
test_data['Label'] = test_data['Label'] - test_data['correction']

label_list = test_data['Label'].to_list()

final_list = []
for i in range(0, len(label_list), 3):    
    label_1 = label_list[i]
    label_2 = label_list[i+1]
    label_3 = label_list[i+2]
    
    if label_1 == 1:
        final_list = final_list + ['A', 'A', 'A']
    elif label_2 == 1:
        final_list = final_list + ['B', 'B', 'B']
    elif label_3 == 1:
        final_list = final_list + ['C','C', 'C']

test_data['option'] = pd.Series(final_list)
# df = df[['FalseSent','option']]
test_data = test_data.drop_duplicates(subset='FalseSent')


test_data = test_data.reset_index()
test_data = test_data[['FalseSent','option']]
test_data = test_data.rename(columns={"FalseSent": "id", "option": "answer"})

sentence_id_list = []
for i in range(1,2001):
    sentence_id_list.append('sentence_' + str(i))
test_data['id'] = pd.Series(sentence_id_list)


test_data





In [0]:
test_data.to_csv('kaggle_predictions.csv', index=False)